# 🕹 Pong

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import gym

In [ ]:
ALFA = 0.05          # learning rate
EPSILON_MIN = 0.01   # valor mínimo de epsilon
EPSILON = 0.7        # valor inicial do epsilon
DECAIMENTO = 0.98    # fator decaímento do epsilon (por episódio)
GAMA = 0.9           # fator de desconto
N_EPISODIOS = 250    # número de episódios

# dicionário dos valores de Q
# chaves: estados; valores: valor atribuido a cada ação
Q = {}

In [ ]:
env = gym.make("pong:turing-easy-v0")

# número total de ações: 3
# 0 = parado; 1 = baixo; 2 = cima
n_acoes = env.action_space.n

print('Número de ações:', n_acoes)

In [ ]:
def discretiza_estado(estado):
    return tuple(round(x/10) for x in estado)

In [ ]:
def salva_tabela(Q, nome = 'model.pickle'):
    with open(nome, 'wb') as pickle_out:
        pickle.dump(Q, pickle_out)

def carrega_tabela(nome = 'model.pickle'):
    with open(nome, 'rb') as pickle_out:
        return pickle.load(pickle_out)

In [ ]:
def escolhe_acao(env, Q, estado, epsilon):
    if estado not in Q.keys(): Q[estado] = [0] * n_acoes

    if np.random.random() < epsilon:
        acao = env.action_space.sample()
    else:
        acao = np.argmax(Q[estado])
    return acao

In [ ]:
def atualiza_q(Q, estado, acao, recompensa, prox_estado):
    # para cada estado ainda não descoberto, iniciamos seu valor como nulo
    if estado not in Q.keys(): Q[estado] = [0] * n_acoes
    if prox_estado not in Q.keys(): Q[prox_estado] = [0] * n_acoes

    # equação de Bellman
    Q[estado][acao] = Q[estado][acao] + ALFA*(recompensa + GAMA*np.max(Q[prox_estado]) - Q[estado][acao])

In [ ]:
def roda_partida(env, Q, renderiza=True):
    estado = env.reset()
    estado = discretiza_estado(estado)
    
    done = False
    retorno = 0
    
    while not done:
        # politica
        acao = escolhe_acao(env, Q, estado, epsilon=0)

        # A ação é tomada e os valores novos são coletados
        # O novo estado é salvo numa nova variavel
        prox_estado, recompensa, done, info = env.step(acao)
        prox_estado = discretiza_estado(prox_estado)

        if renderiza:
            env.render()

        retorno += recompensa
        estado = prox_estado

    print(f'retorno {retorno:.1f},  '
          f'placar {env.score[0]}x{env.score[1]}')
    
    env.close()

In [ ]:
roda_partida(env, Q)

In [ ]:
def treina(env, Q):
    retornos = []      # retorno de cada episódio
    epsilon = EPSILON

    for episodio in range(1, N_EPISODIOS+1):
        estado = env.reset()
        estado = discretiza_estado(estado)
        
        done = False
        retorno = 0
        
        while not done:
            # politica
            acao = escolhe_acao(env, Q, estado, epsilon)

            # A ação é tomada e os valores novos são coletados
            # O novo estado é salvo numa nova variavel
            prox_estado, recompensa, done, info = env.step(acao)
            prox_estado = discretiza_estado(prox_estado)

            atualiza_q(Q, estado, acao, recompensa, prox_estado)

            retorno += recompensa
            estado = prox_estado

        epsilon = max(DECAIMENTO*epsilon, EPSILON_MIN)
        retornos.append(retorno)

        if episodio % 10 == 0:
            salva_tabela(Q)

        print(f'episódio {episodio},  '
              f'retorno {retorno:7.1f},  '
              f'retorno médio (últimos 10 episódios) {np.mean(retornos[-10:]):7.1f},  '
              f'placar {env.score[0]}x{env.score[1]},  '
              f'epsilon: {epsilon:.3f}')
        
    env.close()

In [ ]:
treina(env, Q)

In [ ]:
roda_partida(env, Q)